In [548]:
import sys
sys.path.insert(0, '..')
from importlib import reload
import netket.machine.functions2 as f
import numpy as np

In [549]:
reload(f)
h = f.new_hex(np.array([5,1]))

In [550]:
import scipy.fft as fft
L = 20
states = (f.list_up_states(L) + 1)/2
wave_space = fft.fft(states)
dft_matrix = np.fft.fft(np.eye(L))

In [551]:
a = np.arange(L)
a[3:] = 0
b = 2 ** a
b[3:] = 0
U = np.empty((L,L), dtype=np.int)
for l in range(L):
    U[l] = np.roll(b,-l)

In [591]:
ind = np.where(np.round((np.sqrt(states.dot(U))).dot(dft_matrix[4]),4)==0)[0]
print(ind.shape)

(96,)


In [592]:
states[ind][3]

array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 1.])

In [593]:
%timeit np.round((np.sqrt(states.dot(U))).dot(dft_matrix[1]),4)

157 ms ± 2.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [597]:
np.sqrt(states.dot(U)).shape

(1048576, 20)

In [598]:
states[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [936]:
from numba import njit
@njit
def for_kinetic(x, t, K, normalize=False):
    
    mult_array = np.random.rand(K)
    L = x.shape[1]
    batch_size = x.shape[0]
    out = np.zeros(batch_size, dtype=np.complex128)
    for b in range(batch_size):
        out_ = 0
        for i in range(L):
            temp = 0
            
            for k in range(K):
                temp += x[b,(i+k)%L] * mult_array[k]
            out_ += t[i]*np.sqrt(temp)
#             out_ += t[i]*temp**2
        if normalize:
            out[b] = out_/np.abs(out_)
        else:
            out[b] = out_
    return out


In [1002]:
import scipy.fft as fft
L = 20
states = (f.list_up_states(L) + 1)/2
wave_space = fft.fft(states)
dft_matrix = np.fft.fft(np.eye(L))

In [1003]:
a = np.empty((L,L), dtype=np.int)
for i in range(L):
    for j in range(L):
        a[i,j] = np.where(np.round(for_kinetic(states, dft_matrix[j],i),4)==0)[0].shape[0]

KeyboardInterrupt: 

In [951]:
dft_matrix[2][1]

(0.7660444431189781-0.6427876096865393j)

In [962]:
np.exp(-1j * 2*np.pi*(2/L))

(0.766044443118978-0.6427876096865393j)

In [949]:
a[:,2]

array([262144,   1000,    208,    136,     64,     64,     64,     64,
           64,     64,     64,     64,     64,     64,     64,     64,
           64,     64])

In [1008]:
%timeit indb = np.where(np.round(for_kinetic(states, dft_matrix[2],4),4)==0)[0]

157 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [1005]:
inda = np.where(np.round(for_kinetic(states, dft_matrix[4],4),4)==0)[0]
inda.shape

(16,)

In [774]:
states_finite = states[ind]
%timeit np.round(for_kinetic(states_finite, dft_matrix[4],4,normalize=True),4)

172 ms ± 432 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [1032]:
ind = np.where(np.round(for_kinetic(states, dft_matrix[2],6),4)==0)[0]
print(ind.shape)

(86,)


In [779]:
ind = np.where(np.round(for_kinetic(states, dft_matrix[4],4),4)==0)[0]
print(ind)

[      0   69905  139810  209715  279620  349525  419430  489335  559240
  629145  699050  768955  838860  908765  978670 1048575]


In [780]:
states[341]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1.])

In [781]:
np.round((np.sqrt(states.dot(U))).dot(dft_matrix[2]),4)

array([ 0.    +0.j    , -2.4531-0.6067j, -1.3351+2.1456j, ...,
        0.2079-0.9944j,  1.01  -0.1096j, -0.    +0.j    ])

In [782]:
np.round(for_kinetic(states, dft_matrix[4],3),4)

array([ 0.    +0.j    , -2.4531+0.6067j, -1.3351-2.1456j, ...,
        0.2079+0.9944j,  1.01  +0.1096j, -0.    +0.j    ])

In [1057]:
a = np.random.rand(L)
# a[3:] = 0# b[3:] = 0
U = np.empty((L,L), dtype=np.float64)
for l in range(L):
    U[l] = np.roll(a,-l)
np.sqrt(states[73].dot(U)) * dft_matrix[2]

array([ 1.25988449+0.00000000e+00j,  1.02647402-7.45777029e-01j,
        0.41625051-1.28108735e+00j, -0.34464586-1.06071089e+00j,
       -0.85760086-6.23083496e-01j, -1.2947412 +7.18725747e-17j,
       -0.82487145+5.99304187e-01j, -0.40823811+1.25642772e+00j,
        0.35895501+1.10474993e+00j,  0.69809282+5.07194121e-01j,
        1.34830165+0.00000000e+00j,  0.85550873-6.21563477e-01j,
        0.30374655-9.34835762e-01j, -0.3981587 -1.22540648e+00j,
       -0.95917003-6.96877817e-01j, -1.23479101+0.00000000e+00j,
       -0.88778705+6.45015050e-01j, -0.36542928+1.12467567e+00j,
        0.38557257+1.18667034e+00j,  0.96504593+7.01146910e-01j])

In [1062]:
np.where(np.round(np.sqrt(states.dot(U)).dot(dft_matrix[2]), 4) == 0)[0]

array([      0,   33825,   67650,   69905,  101475,  135300,  139810,
        169125,  202950,  209715,  236775,  270600,  279620,  304425,
        338250,  349525,  372075,  405900,  419430,  439725,  473550,
        489335,  507375,  541200,  559240,  575025,  608850,  629145,
        642675,  676500,  699050,  710325,  744150,  768955,  777975,
        811800,  838860,  845625,  879450,  908765,  913275,  947100,
        978670,  980925, 1014750, 1048575])

In [1063]:
states[33825]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1.])